### SETUP INICIAL DO PROJETO

In [1]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account


# carregar o .env com as credenciais
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")


# detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"


# parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.production-productsubcategory"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"


# configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


#doc: tables_to_process: lista de tabelas que serão processadas
#     output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned

In [2]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

Tabelas a processar: ['desafioadventureworks-446600.raw_data.production-productsubcategory']


In [3]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

raw_data
Tabelas disponíveis:
humanresources-employee
person-address
person-businessentity
person-person
person-stateprovince
production-location
production-product
production-productcategory
production-productcosthistory
production-productinventory
production-productsubcategory
purchasing-purchaseorderdetail
purchasing-purchaseorderheader
purchasing-vendor
sales-creditcard
sales-customer
sales-salesorderdetail
sales-salesorderheader
sales-salesperson
sales-salesterritory
sales-store


# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [4]:
# Setup inicial do df para realizar a EDA 

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [5]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")


print("Todas as tabelas foram processadas com sucesso!")

Processando tabela: desafioadventureworks-446600.raw_data.production-productsubcategory
Lendo os dados do BigQuery...
Tabela production_productsubcategory processada e armazenada com sucesso.
Todas as tabelas foram processadas com sucesso!


In [6]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

Variável criada: production_productsubcategory


In [7]:
# Atribuir o df a uma variável com nome mais simples
production_productsubcategory = df_processados['production_productsubcategory']

print(f"Colunas: {production_productsubcategory.shape[1]}\nLinhas: {production_productsubcategory.shape[0]}")

Colunas: 5
Linhas: 481


In [8]:
production_productsubcategory.head()

,productsubcategoryid,productcategoryid,name,rowguid,modifieddate
0,1,1,Mountain Bikes,2d364ade-264a-433c-b092-4fcbf3804e01,2008-04-30 00:00:00+00:00
1,1,1,Mountain Bikes,2d364ade-264a-433c-b092-4fcbf3804e01,2008-04-30 00:00:00+00:00
2,1,1,Mountain Bikes,2d364ade-264a-433c-b092-4fcbf3804e01,2008-04-30 00:00:00+00:00
3,1,1,Mountain Bikes,2d364ade-264a-433c-b092-4fcbf3804e01,2008-04-30 00:00:00+00:00
4,1,1,Mountain Bikes,2d364ade-264a-433c-b092-4fcbf3804e01,2008-04-30 00:00:00+00:00


In [9]:
# Identificar duplicadas
duplicadas = production_productsubcategory[production_productsubcategory.duplicated(subset=['productsubcategoryid'], keep=False)]

# Verificar se existem duplicadas
if not duplicadas.empty:
    
    duplicadas_ordenadas = duplicadas.sort_values(by=['productsubcategoryid', 'modifieddate'])

    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")

duplicadas ordenadas:
     productsubcategoryid  productcategoryid             name                               rowguid              modifieddate
0                       1                  1   Mountain Bikes  2d364ade-264a-433c-b092-4fcbf3804e01 2008-04-30 00:00:00+00:00
1                       1                  1   Mountain Bikes  2d364ade-264a-433c-b092-4fcbf3804e01 2008-04-30 00:00:00+00:00
2                       1                  1   Mountain Bikes  2d364ade-264a-433c-b092-4fcbf3804e01 2008-04-30 00:00:00+00:00
3                       1                  1   Mountain Bikes  2d364ade-264a-433c-b092-4fcbf3804e01 2008-04-30 00:00:00+00:00
4                       1                  1   Mountain Bikes  2d364ade-264a-433c-b092-4fcbf3804e01 2008-04-30 00:00:00+00:00
..                    ...                ...              ...                                   ...                       ...
440                    37                  4  Tires and Tubes  3c17c9ae-e906-48b4-bdd3-60e28d47d

In [10]:
# Remover duplicadas* 
production_productsubcategory = production_productsubcategory.drop_duplicates(subset=['productsubcategoryid'], keep='last')

print(f"Linhas após remover duplicadas (baseando-se na última 'modifieddate'): {len(production_productsubcategory)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicadas = production_productsubcategory.copy()


#doc: bkp = backup (cópia)
#doc*: mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
#      Importante, pois se houver erro na ingestão (duplicação), mantém os dados íntegros.

Linhas após remover duplicadas (baseando-se na última 'modifieddate'): 37


In [11]:
# Ordenar e exibir o df por 'productsubcategoryid'
production_productsubcategory = production_productsubcategory.sort_values(by=['productsubcategoryid'])

print(production_productsubcategory)

     productsubcategoryid  productcategoryid               name                               rowguid              modifieddate
444                     1                  1     Mountain Bikes  2d364ade-264a-433c-b092-4fcbf3804e01 2008-04-30 00:00:00+00:00
445                     2                  1         Road Bikes  000310c0-bcc8-42c4-b0c3-45ae611af06b 2008-04-30 00:00:00+00:00
446                     3                  1      Touring Bikes  02c5061d-ecdc-4274-b5f1-e91d76bc3f37 2008-04-30 00:00:00+00:00
447                     4                  2         Handlebars  3ef2c725-7135-4c85-9ae6-ae9a3bdd9283 2008-04-30 00:00:00+00:00
448                     5                  2    Bottom Brackets  a9e54089-8a1e-4cf5-8646-e3801f685934 2008-04-30 00:00:00+00:00
449                     6                  2             Brakes  d43ba4a3-ef0d-426b-90eb-4be4547dd30c 2008-04-30 00:00:00+00:00
450                     7                  2             Chains  e93a7231-f16c-4b0f-8c41-c73fdec62da0 20

In [12]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in production_productsubcategory.columns:   
    missing_rows = production_productsubcategory[production_productsubcategory[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

Coluna 'productsubcategoryid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'productsubcategoryid'.

Coluna 'productcategoryid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'productcategoryid'.

Coluna 'name': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'name'.

Coluna 'rowguid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'rowguid'.

Coluna 'modifieddate': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'modifieddate'.



In [13]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = production_productsubcategory.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

Valores únicos incluindo NaN:
productsubcategoryid    37
productcategoryid        4
name                    37
rowguid                 37
modifieddate             1
dtype: int64


In [14]:
#verificar informações do df
production_productsubcategory.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 444 to 480
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   productsubcategoryid  37 non-null     Int64              
 1   productcategoryid     37 non-null     Int64              
 2   name                  37 non-null     object             
 3   rowguid               37 non-null     object             
 4   modifieddate          37 non-null     datetime64[us, UTC]
dtypes: Int64(2), datetime64[us, UTC](1), object(2)
memory usage: 1.8+ KB


In [15]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['production_productsubcategory'] = production_productsubcategory

In [16]:
# Padronizar colunas com valores textuais
production_productsubcategory['name'] = production_productsubcategory['name'].str.strip().str.upper()
production_productsubcategory['rowguid'] = production_productsubcategory['rowguid'].str.strip().str.upper()

print(production_productsubcategory.head())

#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

     productsubcategoryid  productcategoryid             name                               rowguid              modifieddate
444                     1                  1   MOUNTAIN BIKES  2D364ADE-264A-433C-B092-4FCBF3804E01 2008-04-30 00:00:00+00:00
445                     2                  1       ROAD BIKES  000310C0-BCC8-42C4-B0C3-45AE611AF06B 2008-04-30 00:00:00+00:00
446                     3                  1    TOURING BIKES  02C5061D-ECDC-4274-B5F1-E91D76BC3F37 2008-04-30 00:00:00+00:00
447                     4                  2       HANDLEBARS  3EF2C725-7135-4C85-9AE6-AE9A3BDD9283 2008-04-30 00:00:00+00:00
448                     5                  2  BOTTOM BRACKETS  A9E54089-8A1E-4CF5-8646-E3801F685934 2008-04-30 00:00:00+00:00


In [17]:
# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
 
    output_table = f"{output_dataset}.{table_name}"
   
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")

Tabela production_productsubcategory exportada com sucesso para desafioadventureworks-446600.raw_data_cleaned.production_productsubcategory.
